## Sarcasm Detection – Dataset Preparation & Annotation Workflow

This notebook prepares the dataset for sarcasm detection experiments.
It includes dataset merging, annotation setup, quality control, and final dataset preparation for modeling.

In [3]:
# Used Libraries

import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import re
import sys
import csv
from urllib.parse import urlparse
import time
import json
import random
from dotenv import load_dotenv
import anthropic

load_dotenv()

False

### Constants

In [4]:
BASE_DIR = Path().resolve().parents[1]

DATA_DIR = BASE_DIR / "data"
PREPROCESSED_DF = DATA_DIR / "preprocessed_kosovo_news.csv"
PREPARED_FOR_ANNOTATION_DF = DATA_DIR / "annotation_kungulli_3k.csv"
ANNOTATED_DF = DATA_DIR / "annotation_kungulli_3k_with_sarcasm.csv"
PREPARED_LABELED_DF = DATA_DIR / "sarcasm_detection_dataset_v1.csv"

In [35]:
print(PREPROCESSED_DF)
df = pd.read_csv(PREPROCESSED_DF)
df.head()

/Users/bleronaidrizi/Sources/Master_Tema_e_Diplomes/Punimi/Sarcasm-Detection-Albanian-News-Dataset/data/preprocessed_kosovo_news.csv


,content,title,category,source,text
0,Kur Beatrice Borromeo u martua me Pierre Casir...,As Kate as Meghan; ja cila është princesha më ...,Fun;Argëtim,Lajmi,As Kate as Meghan; ja cila është princesha më ...
1,Gjatë kontrollit të automjetit zyrtarët polico...,"I kapen 10 kg substanca narkotike në BMW X5, a...",Lajme;Nacionale,Lajmi,"I kapen 10 kg substanca narkotike në BMW X5, a..."
2,Enca Haxhia njihet si një ndër këngëtaret më s...,Enca e quan jetë pushimin në plazh me poza në ...,Entertainment,Lajmi,Enca e quan jetë pushimin në plazh me poza në ...
3,Gurët në veshka janë depozitime minerale që fo...,Gurët në veshka – Kurat natyrale dhe si t’i pë...,Lifestyle;Shëndeti,Lajmi,Gurët në veshka – Kurat natyrale dhe si t’i pë...
4,Në vendim thuhet se Zyra e Prokurorit të Speci...,Trupi gjykues ua vazhdon paraburgimin Nasim Ha...,Lajme;Nacionale,Lajmi,Trupi gjykues ua vazhdon paraburgimin Nasim Ha...


### 1. Data Sampling

- Load preprocessed dataset
- Merge Kungulli sample with other sources
- Create final annotation dataset

In [ ]:
TARGET_N = 3000
CHUNKSIZE = 50_000
RANDOM_STATE = 42
# DEBUG_LIMIT = 4   # None për full run

SATIRE_DOMAINS = {"kungulli.com", "www.kungulli.com"}

csv.field_size_limit(sys.maxsize)

# =========================
# HELPERS
# =========================
def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.astype(str).str.strip().str.lower()
    return df

def get_output_path(data_path: str, out_file: str) -> str:
    folder = os.path.dirname(os.path.abspath(data_path))
    return os.path.join(folder, out_file)

def extract_domain(url: str) -> str:
    if pd.isna(url):
        return ""
    s = str(url).strip()
    if not s:
        return ""
    if not re.match(r"^https?://", s, flags=re.IGNORECASE):
        s = "https://" + s
    try:
        host = urlparse(s).netloc.lower()
        host = host.split(":")[0]
        return host
    except Exception:
        return ""

def read_chunks(path: str):
    try:
        return pd.read_csv(
            path,
            chunksize=CHUNKSIZE,
            sep=",",
            engine="c",
            on_bad_lines="skip",
        )
    except Exception:
        return pd.read_csv(
            path,
            chunksize=CHUNKSIZE,
            sep=",",
            engine="python",
            on_bad_lines="skip",
        )

def ensure_source_column(df: pd.DataFrame) -> pd.DataFrame:
    cols = set(df.columns)

    if "source" in cols:
        df["source"] = df["source"].astype(str).str.strip().str.lower()
        return df

    if "url" in cols:
        df["source"] = df["url"].apply(extract_domain)
        return df

    raise ValueError(
        "The dataset has neither 'source' nor 'url'. I can't find pumpkin without either of them."
    )

def filter_kungulli(df: pd.DataFrame) -> pd.DataFrame:
    df = ensure_source_column(df)
    df["source"] = df["source"].astype(str).str.lower().str.strip()

    mask = df["source"].isin(SATIRE_DOMAINS) | df["source"].str.contains("kungulli", na=False)
    return df[mask].copy()

# =========================
# MAIN
# =========================
def run():
    df0 = pd.read_csv(PREPROCESSED_DF, nrows=1)
    df0 = normalize_cols(df0)
    print("Header:", list(df0.columns))

    kungulli_rows = []
    other_rows = []

    for chunk in read_chunks(PREPROCESSED_DF):
        chunk = normalize_cols(chunk)

        try:
            k = filter_kungulli(chunk)
        except ValueError as e:
            print("❌", str(e))
            return

        # kungulli
        if len(k) > 0:
            kungulli_rows.append(k)

        # not kungulli
        chunk = ensure_source_column(chunk)
        chunk["source"] = chunk["source"].astype(str).str.lower().str.strip()
        mask_other = ~(
            chunk["source"].isin(SATIRE_DOMAINS) |
            chunk["source"].str.contains("kungulli", na=False)
        )
        o = chunk[mask_other]
        if len(o) > 0:
            other_rows.append(o)

    if not kungulli_rows:
        print("No kungulli found.")
        return

    df_k = pd.concat(kungulli_rows, ignore_index=True).drop_duplicates()
    df_o = pd.concat(other_rows, ignore_index=True).drop_duplicates()

    print("Kungulli found:", len(df_k))
    print("Other sources found:", len(df_o))

    # -------------------------
    # Build 3K total
    # -------------------------
    needed_from_other = TARGET_N - len(df_k)

    if needed_from_other <= 0:
        df_out = df_k.sample(n=TARGET_N, random_state=RANDOM_STATE)
    else:
        df_o_sample = df_o.sample(
            n=min(needed_from_other, len(df_o)),
            random_state=RANDOM_STATE
        )
        df_out = pd.concat([df_k, df_o_sample], ignore_index=True)

    # Shuffle final
    df_out = df_out.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

    df_out["is_sarcasm(1|0|?)"] = ""

    out_path = get_output_path(PREPROCESSED_DF, PREPARED_FOR_ANNOTATION_DF)
    df_out.to_csv(out_path, index=False, encoding="utf-8")

    print("\n✅ Saved:", out_path)
    print("Total rows:", len(df_out))
    print("Kungulli in final:", sum(df_out["source"].str.contains("kungulli")))

run()

Header: ['content', 'title', 'category', 'source', 'text']
Kungulli found: 694
Other sources found: 1454473

✅ Saved: /Users/fatlumberisha/Blerona/Sarcasm-Detection-Albanian-News-Dataset/data/annotation_kungulli_3k.csv
Total rows: 3000
Kungulli in final: 694


### 2. Claude-Version

In [ ]:
MODEL = os.getenv("CLAUDE_MODEL", "claude-opus-4-6")
API_KEY = os.getenv("ANTHROPIC_API_KEY")

MAX_CHARS = int(os.getenv("MAX_CHARS", "2500"))
SLEEP_BETWEEN = float(os.getenv("SLEEP_BETWEEN", "0.1"))
MAX_RETRIES = 5

# ---------------------------
# Helpers
# ---------------------------
def truncate_text(s: str, max_chars: int) -> str:
    s = str(s).strip()
    if len(s) <= max_chars:
        return s
    return s[: max_chars - 50] + "\n\n[TRUNCATED]"

def backoff_sleep(attempt: int) -> None:
    base = min(2 ** attempt, 20)
    jitter = random.uniform(0, 0.5)
    time.sleep(base + jitter)

def extract_json(text: str) -> dict:
    text = (text or "").strip()
    try:
        return json.loads(text)
    except Exception:
        pass

    start = text.find("{")
    end = text.rfind("}")
    if start != -1 and end != -1 and end > start:
        return json.loads(text[start:end + 1])

    raise ValueError(f"No valid JSON found in model output: {text[:200]}...")

def read_dataset(path: Path) -> pd.DataFrame:
    # robust encoding fallback (du hattest UTF-8 Probleme)
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin1"):
        try:
            return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding="utf-8", encoding_errors="replace")

def detect_sarcasm(client: anthropic.Anthropic, text: str) -> int:
    system = (
        "You are a sarcasm detector for Albanian-language news texts.\n"
        "Label sarcasm as 1 if the text uses irony, mockery, sarcasm, or clearly says the opposite of intent.\n"
        "Otherwise label 0.\n"
        "If uncertain, choose 0.\n"
        "Return ONLY valid JSON exactly in this format:\n"
        '{"sarcasm": 0 or 1, "confidence": 0.0-1.0}\n'
        "No extra text."
    )

    user_text = truncate_text(text, MAX_CHARS)

    for attempt in range(MAX_RETRIES):
        try:
            msg = client.messages.create(
                model=MODEL,
                max_tokens=80,
                temperature=0.0,
                system=system,
                messages=[{"role": "user", "content": user_text}],
            )
            # msg.content ist Liste von Content-Blöcken; typischerweise 1 Textblock
            out_text = "".join(block.text for block in msg.content if getattr(block, "type", "") == "text")
            data = extract_json(out_text)
            return int(data.get("sarcasm", 0))
        except Exception:
            if attempt == MAX_RETRIES - 1:
                raise
            backoff_sleep(attempt)

    return 0

def main():
    if not API_KEY:
        raise SystemExit("Missing ANTHROPIC_API_KEY in .env")

    print("INPUT_CSV:", PREPARED_FOR_ANNOTATION_DF)
    print("OUTPUT_CSV:", ANNOTATED_DF)

    df = read_dataset(PREPARED_FOR_ANNOTATION_DF)

    if "content" not in df.columns:
        raise ValueError("The column 'content' does not exist.")

    # Create sarcasm column if missing
    if "sarcasm" not in df.columns:
        df["sarcasm"] = None

    client = anthropic.Anthropic(api_key=API_KEY)

    total = len(df)
    processed_since_save = 0

    for i in range(total):
        # Skip if already processed
        if pd.notna(df.loc[i, "sarcasm"]):
            continue

        text = str(df.loc[i, "content"]).strip()
        if not text:
            df.loc[i, "sarcasm"] = 0
            continue

        try:
            label = detect_sarcasm(client, text)
        except Exception as e:
            print(f"Error at row {i}: {e}")
            df.loc[i, "sarcasm"] = 0
            continue

        df.loc[i, "sarcasm"] = label
        processed_since_save += 1

        # Save every 100 rows
        if processed_since_save >= 100:
            df.to_csv(ANNOTATED_DF, index=False, encoding="utf-8")
            print(f"💾 Saved progress at row {i}")
            processed_since_save = 0

        if i % 50 == 0:
            print(f"Processed {i}/{total}")

        time.sleep(SLEEP_BETWEEN)

    # Final save
    df.to_csv(ANNOTATED_DF, index=False, encoding="utf-8")
    print("✅ Done. All rows processed.")

main()
client = anthropic.Anthropic(api_key=API_KEY)

for m in client.models.list():
    print(m.id)

INPUT_CSV: /Users/fatlumberisha/Blerona/Sarcasm-Detection-Albanian-News-Dataset/data/annotation_kungulli_3k.csv
OUTPUT_CSV: /Users/fatlumberisha/Blerona/Sarcasm-Detection-Albanian-News-Dataset/data/annotation_kungulli_3k_with_sarcasm.csv
Processed 0/3000
Processed 50/3000
💾 Saved progress at row 99
Processed 100/3000
Processed 150/3000
💾 Saved progress at row 199
Processed 200/3000
Processed 250/3000
💾 Saved progress at row 299
Processed 300/3000
Processed 350/3000
💾 Saved progress at row 399
Processed 400/3000
Processed 450/3000
💾 Saved progress at row 499
Processed 500/3000
Processed 550/3000
💾 Saved progress at row 599
Processed 600/3000
Processed 650/3000
💾 Saved progress at row 699
Processed 700/3000
⚠️ Error at row 702: No valid JSON found in model output: ...
Processed 750/3000
💾 Saved progress at row 800
Processed 800/3000
Processed 850/3000
💾 Saved progress at row 900
Processed 900/3000
Processed 950/3000
💾 Saved progress at row 1000
Processed 1000/3000
Processed 1050/3000
💾 S

# OPENAI

In [2]:
# ============================================================
# Sarcasm labeling (OpenAI) - RESUME FRIENDLY + CLEAN DATASET
# - Reads .env (override=True)
# - Keeps ONLY: content, category, (optional source), is_sarcasm, (optional sarcasm_confidence)
# - Labels ONLY rows where is_sarcasm is missing
# - Saves progress every SAVE_EVERY rows -> can stop/restart anytime
# - Builds a balanced dataset at the end
# ============================================================

import os
import re
import json
import time
import random
import csv
import sys
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

csv.field_size_limit(sys.maxsize)

# -----------------------
# ENV (.env)
# -----------------------
load_dotenv(override=True)

# -----------------------
# CONFIG
# -----------------------
MODEL = os.getenv("OPENAI_MODEL", "gpt-4.1-nano")  # cheap; alt: gpt-4o-mini
API_KEY = os.getenv("OPENAI_API_KEY")              # ✅ from .env (NO hardcode)

MAX_CHARS = int(os.getenv("MAX_CHARS", "2000"))
SLEEP_BETWEEN = float(os.getenv("SLEEP_BETWEEN", "0.15"))
MAX_RETRIES = int(os.getenv("MAX_RETRIES", "5"))
SAVE_EVERY = int(os.getenv("SAVE_EVERY", "100"))

TARGET_SARCASM = int(os.getenv("TARGET_SARCASM", "1500"))
TARGET_NON = int(os.getenv("TARGET_NON", "1500"))
RANDOM_SEED = int(os.getenv("RANDOM_SEED", "42"))

KEEP_SOURCE = True          # keep 'source' column or not
KEEP_CONFIDENCE = True      # keep 'sarcasm_confidence' column or not

# Repo root resolver (works from notebooks/)
REPO_ROOT = Path.cwd()
while REPO_ROOT != REPO_ROOT.parent and not (REPO_ROOT / "data").exists():
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"
INPUT_FILE = DATA_DIR / "preprocessed_kosovo_news.csv"
PROGRESS_FILE = DATA_DIR / "preprocessed_kosovo_news_with_is_sarcasm_v1.csv"
FINAL_BALANCED_FILE = DATA_DIR / f"sarcasm_balanced_{TARGET_SARCASM}_{TARGET_NON}_v1.csv"

# -----------------------
# Helpers
# -----------------------
def truncate_text(s: str, max_chars: int) -> str:
    s = str(s or "").strip()
    if len(s) <= max_chars:
        return s
    return s[: max_chars - 50] + "\n\n[TRUNCATED]"

def backoff_sleep(attempt: int) -> None:
    base = min(2 ** attempt, 20)
    jitter = random.uniform(0, 0.5)
    time.sleep(base + jitter)

def extract_json_loose(text: str) -> dict:
    text = (text or "").strip()
    try:
        return json.loads(text)
    except Exception:
        pass
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if not m:
        raise ValueError(f"No JSON found: {text[:200]}")
    return json.loads(m.group(0))

def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin1"):
        try:
            return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding="utf-8", encoding_errors="replace")

def detect_sarcasm_openai(client: OpenAI, text: str) -> tuple[int, float]:
    system = (
        "You are a sarcasm detector for Albanian-language news texts.\n"
        "Return ONLY valid JSON in this exact format:\n"
        '{"sarcasm": 0 or 1, "confidence": 0.0-1.0}\n'
        "Rules:\n"
        "- sarcasm=1 if there is irony/mockery/sarcasm or meaning is opposite of literal.\n"
        "- sarcasm=0 otherwise.\n"
        "- If uncertain, choose 0.\n"
        "No extra text."
    )

    user_text = truncate_text(text, MAX_CHARS)

    for attempt in range(MAX_RETRIES):
        try:
            resp = client.responses.create(
                model=MODEL,
                input=[
                    {"role": "system", "content": system},
                    {"role": "user", "content": user_text},
                ],
                temperature=0.0,
                max_output_tokens=80,
            )
            out_text = (getattr(resp, "output_text", None) or "").strip()
            data = extract_json_loose(out_text)
            sarcasm = int(data.get("sarcasm", 0))
            conf = float(data.get("confidence", 0.0))
            return sarcasm, conf
        except Exception:
            if attempt == MAX_RETRIES - 1:
                raise
            backoff_sleep(attempt)

    return 0, 0.0

# -----------------------
# MAIN
# -----------------------
def main():
    if not API_KEY:
        raise SystemExit("Missing OPENAI_API_KEY. Put it in .env and restart kernel.")

    random.seed(RANDOM_SEED)
    client = OpenAI(api_key=API_KEY)

    # 1) Load progress or input
    if PROGRESS_FILE.exists():
        df = read_csv_robust(PROGRESS_FILE)
        print("Resuming existing progress file:", PROGRESS_FILE)
    else:
        df = read_csv_robust(INPUT_FILE)
        print("Starting from input file:", INPUT_FILE)

    # 2) Normalize / ensure needed columns
    df.columns = [c.strip().lower() for c in df.columns]

    if "content" not in df.columns:
        raise ValueError("Column 'content' not found.")

    if "category" not in df.columns:
        df["category"] = pd.NA

    if KEEP_SOURCE and "source" not in df.columns:
        df["source"] = pd.NA

    if "is_sarcasm" not in df.columns:
        df["is_sarcasm"] = pd.NA

    if KEEP_CONFIDENCE and "sarcasm_confidence" not in df.columns:
        df["sarcasm_confidence"] = pd.NA

    # 3) Keep ONLY the small columns (removes title/text/etc permanently)
    keep_cols = ["content", "category", "is_sarcasm"]
    if KEEP_SOURCE:
        keep_cols.append("source")
    if KEEP_CONFIDENCE:
        keep_cols.append("sarcasm_confidence")

    df = df[keep_cols].copy()

    # 4) Clean content + normalize labels
    df["content"] = df["content"].fillna("").astype(str).str.strip()
    df = df[df["content"] != ""].reset_index(drop=True)

    # normalize existing labels to int where possible
    labeled_mask = df["is_sarcasm"].isin([0, 1, "0", "1", 0.0, 1.0])
    df.loc[labeled_mask, "is_sarcasm"] = df.loc[labeled_mask, "is_sarcasm"].astype(int)

    current_ones = int((df["is_sarcasm"] == 1).sum())
    current_zeros = int((df["is_sarcasm"] == 0).sum())
    print(f"Already labeled -> 1s:{current_ones}, 0s:{current_zeros}, total:{len(df)}")

    # 5) IMPORTANT: only label rows where is_sarcasm is missing (resume-friendly)
    missing_idx = df.index[~df["is_sarcasm"].isin([0, 1])].tolist()
    print("Rows to label (missing):", len(missing_idx))

    processed_since_save = 0

    for n, i in enumerate(missing_idx, start=1):
        if current_ones >= TARGET_SARCASM and current_zeros >= TARGET_NON:
            print(f"✅ Stop: reached targets 1s={TARGET_SARCASM} and 0s={TARGET_NON}")
            break

        text = df.at[i, "content"]
        if not text:
            df.at[i, "is_sarcasm"] = 0
            if KEEP_CONFIDENCE:
                df.at[i, "sarcasm_confidence"] = 0.0
            current_zeros += 1
            continue

        try:
            label, conf = detect_sarcasm_openai(client, text)
        except Exception as e:
            print(f"Error at row {i}: {e} -> set 0")
            label, conf = 0, 0.0

        df.at[i, "is_sarcasm"] = int(label)
        if KEEP_CONFIDENCE:
            df.at[i, "sarcasm_confidence"] = float(conf)

        if label == 1:
            current_ones += 1
        else:
            current_zeros += 1

        processed_since_save += 1

        if processed_since_save >= SAVE_EVERY:
            df.to_csv(PROGRESS_FILE, index=False, encoding="utf-8")
            print(f"💾 Saved | last_row={i} | 1s:{current_ones} 0s:{current_zeros} | labeled_in_run:{n}")
            processed_since_save = 0

        if n % 50 == 0:
            print(f"Labeled {n}/{len(missing_idx)} | last_row={i} | 1s:{current_ones} 0s:{current_zeros}")

        time.sleep(SLEEP_BETWEEN)

    # final save
    df.to_csv(PROGRESS_FILE, index=False, encoding="utf-8")
    print("✅ Progress saved:", PROGRESS_FILE)

    # 6) Build balanced dataset (only from labeled rows)
    labeled_df = df[df["is_sarcasm"].isin([0, 1])].copy()
    labeled_df["is_sarcasm"] = labeled_df["is_sarcasm"].astype(int)
    labeled_df = labeled_df.drop_duplicates(subset=["content"]).reset_index(drop=True)

    ones = labeled_df[labeled_df["is_sarcasm"] == 1]
    zeros = labeled_df[labeled_df["is_sarcasm"] == 0]

    n1 = min(TARGET_SARCASM, len(ones))
    n0 = min(TARGET_NON, len(zeros))

    balanced = pd.concat(
        [
            ones.sample(n=n1, random_state=RANDOM_SEED),
            zeros.sample(n=n0, random_state=RANDOM_SEED),
        ],
        ignore_index=True,
    ).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    # final columns (what you asked)
    final_cols = ["content", "category", "is_sarcasm"]
    if KEEP_SOURCE:
        final_cols.append("source")
    if KEEP_CONFIDENCE:
        final_cols.append("sarcasm_confidence")

    balanced = balanced[final_cols]
    balanced.to_csv(FINAL_BALANCED_FILE, index=False, encoding="utf-8")
    print("✅ Balanced dataset saved:", FINAL_BALANCED_FILE)
    print("Balanced counts:", balanced["is_sarcasm"].value_counts().to_dict())


if __name__ == "__main__":
    main()

Resuming existing progress file: /Users/bleronaidrizi/Sources/Master_Tema_e_Diplomes/Punimi/Sarcasm-Detection-Albanian-News-Dataset/data/preprocessed_kosovo_news_with_is_sarcasm_v1.csv
Already labeled -> 1s:751, 0s:116949, total:120000
Rows to label (missing): 2300
Labeled 50/2300 | last_row=117749 | 1s:751 0s:116999
💾 Saved | last_row=117799 | 1s:751 0s:117049 | labeled_in_run:100
Labeled 100/2300 | last_row=117799 | 1s:751 0s:117049
Labeled 150/2300 | last_row=117849 | 1s:751 0s:117099
💾 Saved | last_row=117899 | 1s:751 0s:117149 | labeled_in_run:200
Labeled 200/2300 | last_row=117899 | 1s:751 0s:117149
Labeled 250/2300 | last_row=117949 | 1s:751 0s:117199
💾 Saved | last_row=117999 | 1s:751 0s:117249 | labeled_in_run:300
Labeled 300/2300 | last_row=117999 | 1s:751 0s:117249
Labeled 350/2300 | last_row=118049 | 1s:753 0s:117297
💾 Saved | last_row=118099 | 1s:753 0s:117347 | labeled_in_run:400
Labeled 400/2300 | last_row=118099 | 1s:753 0s:117347
Labeled 450/2300 | last_row=118149 | 1s

In [ ]:
from pathlib import Path
import pandas as pd

# -----------------------
# PATHS
# -----------------------
REPO_ROOT = Path.cwd()
while REPO_ROOT != REPO_ROOT.parent and not (REPO_ROOT / "data").exists():
    REPO_ROOT = REPO_ROOT.parent

DATA_DIR = REPO_ROOT / "data"

PRED_FILE = DATA_DIR / "annotation_kungulli_3k_with_sarcasm.csv"
PROGRESS_FILE = DATA_DIR / "preprocessed_kosovo_news_with_is_sarcasm_v1.csv"

OUT_MERGED_ALL = DATA_DIR / "sarcasm_merged_all_labeled.csv"
OUT_BALANCED = DATA_DIR / "sarcasm_balanced_1500_1500_merged.csv"

TARGET_1 = 1500
TARGET_0 = 1500
SEED = 42

# -----------------------
# HELPERS
# -----------------------
def read_csv_robust(path: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin1"):
        try:
            return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding=enc)
        except UnicodeDecodeError:
            continue
    return pd.read_csv(path, engine="python", on_bad_lines="skip", encoding="utf-8", encoding_errors="replace")

def normalize(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]

    # content
    if "content" not in df.columns:
        raise ValueError("Missing column 'content' in one of the files.")
    df["content"] = df["content"].fillna("").astype(str).str.strip()
    df = df[df["content"] != ""].copy()

    # category (optional)
    if "category" not in df.columns:
        df["category"] = pd.NA

    # label column name differences
    if "is_sarcasm" not in df.columns:
        if "sarcasm" in df.columns:
            df["is_sarcasm"] = df["sarcasm"]
        elif "labels" in df.columns:
            df["is_sarcasm"] = df["labels"]
        else:
            raise ValueError("Missing label column: expected 'is_sarcasm' (or sarcasm/labels).")

    # normalize label values to int 0/1
    df["is_sarcasm"] = df["is_sarcasm"].map(lambda x: str(x).strip() if pd.notna(x) else x)
    df = df[df["is_sarcasm"].isin(["0", "1", 0, 1, 0.0, 1.0])].copy()
    df["is_sarcasm"] = df["is_sarcasm"].astype(int)

    # keep only needed cols
    df = df[["content", "category", "is_sarcasm"]].copy()
    return df

# -----------------------
# LOAD
# -----------------------
df_pred = normalize(read_csv_robust(PRED_FILE))
df_prog = normalize(read_csv_robust(PROGRESS_FILE))

print("pred counts:", df_pred["is_sarcasm"].value_counts().to_dict())
print("prog counts:", df_prog["is_sarcasm"].value_counts().to_dict())

# -----------------------
# MERGE + DEDUPE
# -----------------------
merged = pd.concat([df_pred, df_prog], ignore_index=True)

# normalize whitespace just in case
merged["content"] = merged["content"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()

# drop duplicates by content (keeps first)
merged = merged.drop_duplicates(subset=["content"]).reset_index(drop=True)

print("merged counts:", merged["is_sarcasm"].value_counts().to_dict())

# -----------------------
# BALANCE (1500/1500)
# -----------------------
ones = merged[merged["is_sarcasm"] == 1]
zeros = merged[merged["is_sarcasm"] == 0]

n1 = min(TARGET_1, len(ones))
n0 = min(TARGET_0, len(zeros))

balanced = pd.concat(
    [
        ones.sample(n=n1, random_state=SEED),
        zeros.sample(n=n0, random_state=SEED),
    ],
    ignore_index=True,
).sample(frac=1, random_state=SEED).reset_index(drop=True)

print("balanced counts:", balanced["is_sarcasm"].value_counts().to_dict())
balanced.to_csv(OUT_BALANCED, index=False, encoding="utf-8")
print("Saved balanced dataset:", OUT_BALANCED)

pred counts: {0: 2259, 1: 741}
prog counts: {}
merged counts: {0: 2259, 1: 741}
balanced counts: {0: 1500, 1: 741}
Saved balanced dataset: /Users/bleronaidrizi/Sources/Master_Tema_e_Diplomes/Punimi/Sarcasm-Detection-Albanian-News-Dataset/data/sarcasm_balanced_1500_1500_merged.csv
